In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras # official high level api, goto for tensorflow NN
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [8]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()
print(x_train.shape)

# flatten and normalize the data to 0-1
x_train = x_train.reshape(-1,28*28).astype("float32")/255.0 # numpy array
x_test = x_test.reshape(-1,28*28).astype("float32")/255.0 

print(x_train.shape)

(60000, 28, 28)
(60000, 784)


In [14]:
# Sequential API (Very convenient, not flexible) one input to one output

model = keras.Sequential(
    [
        keras.Input(shape=(28*28)), # let you do summary before fit
        layers.Dense(512,activation='relu'),
        layers.Dense(256,activation='relu'),
        layers.Dense(10),
        
    ]
)

print(model.summary())

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"],
)

model.fit(x_train,y_train,batch_size=32, epochs=5,verbose=2)
model.evaluate(x_test,y_test,batch_size=32,verbose=2)




Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
1875/1875 - 4s - loss: 0.1859 - accuracy: 0.9435
Epoch 2/5
1875/1875 - 4s - loss: 0.0808 - accuracy: 0.9752
Epoch 3/5
1875/1875 - 4s - loss: 0.0549 - accuracy: 0.9826
Epoch 4/5
1875/1875 - 4s - loss: 0.0407 - accuracy: 0.9864
Epoch 5/5
1875/1875 - 4s - loss: 0.0318 - accuracy: 0.9895
313/313 - 0s - loss: 0.0782 - accuracy: 0.9780


[0.07815806567668915, 0.9779999852180481]

In [16]:
# alterntive way to setup the model that is more flexible for debugging
# use sequential if you can

model = keras.Sequential()
model.add(keras.Input(shape=(784)))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(256, activation="relu", name="my_layer"))
model.add(layers.Dense(10))

print(model.summary())

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"],
)

model.fit(x_train,y_train,batch_size=32, epochs=5,verbose=2)
model.evaluate(x_test,y_test,batch_size=32,verbose=2)


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 512)               401920    
_________________________________________________________________
my_layer (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_18 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
1875/1875 - 5s - loss: 0.1858 - accuracy: 0.9436
Epoch 2/5
1875/1875 - 4s - loss: 0.0797 - accuracy: 0.9749
Epoch 3/5
1875/1875 - 4s - loss: 0.0536 - accuracy: 0.9831
Epoch 4/5
1875/1875 - 4s - loss: 0.0401 - accuracy: 0.9873
Epoch 5/5
1875/1875 - 4s - loss: 0.0351 - accuracy: 0.9890
313/313 - 0s - loss: 0.0696 - accuracy: 0.9795


[0.0696335956454277, 0.9794999957084656]

In [19]:
# functional API (more flexible, multiple input and output)


inputs = keras.Input(shape=(784))
x = layers.Dense(512, activation="relu", name="first_layer")(inputs)
x = layers.Dense(256, activation="relu", name="second_layer")(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"],
)

# Tensorboard

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir = "tb_callback_dir",
    histogram_freq = 1,
)

model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2,callbacks=[tensorboard_callback])
model.evaluate(x_test, y_test, batch_size=32, verbose=2)


Epoch 1/5
1875/1875 - 5s - loss: 0.1837 - accuracy: 0.9449
Epoch 2/5
1875/1875 - 5s - loss: 0.0796 - accuracy: 0.9755
Epoch 3/5
1875/1875 - 5s - loss: 0.0559 - accuracy: 0.9826
Epoch 4/5
1875/1875 - 5s - loss: 0.0413 - accuracy: 0.9867
Epoch 5/5
1875/1875 - 5s - loss: 0.0335 - accuracy: 0.9897
313/313 - 0s - loss: 0.0737 - accuracy: 0.9801


[0.07372310757637024, 0.9800999760627747]

In [18]:
# we can debug each of the layers
# https://www.youtube.com/watch?v=pAhPiF3yiXI&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=3 
# after 17:31s